# Model Classes

> Model classes

In [ ]:
#| default_exp utils.model_classes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
from pathlib import Path
from collections.abc import Sequence
from typing import (
  Callable, Generic, 
  TypeVar, Union, 
  Protocol, Dict,
  TypeAlias)
import xarray as xr
import types

In [ ]:
#| export
class _Model(ABC):
  
  @abstractmethod
  def predict(self, x: xr.Dataset) -> xr.Dataset:
    pass
  
  @abstractmethod
  def contributions(self, x: xr.Dataset) -> xr.Dataset:
    pass
  
  def __repr__(self) -> str:
    return f"Model"

In [ ]:
#| export
Budget = Union[Dict[str, float], xr.Dataset]

In [ ]:
#| export
class BaseBudgetModel(_Model):
    """
    Abstract class for all models
    """
    _FUNCTION_MODULE_NAME = "functions.py"
    
    def __init__(
      self, 
      model_name: str, # Name used to identify the model
      model_kpi: str, # Key performance indicator output by the model predict
      model_path: str|Path, # Path to the model artifact
    ):
        self.model_name: str = model_name
        self.model_kpi: str = model_kpi
        self.model_path: Path = model_path if isinstance(model_path, Path) else Path(model_path)
        self._model = self._get_model_loader()(model_path)
        self._budget_to_data = self._get_budget_to_data()

    def _get_model_loader(self) -> Callable[str|Path, _Model]:
        """
        Get the function to load the model from the path
        """
        from importlib.util import spec_from_file_location, module_from_spec
        spec = spec_from_file_location(self._FUNCTION_MODULE_NAME.replace(".py", ""), self.model_path / self._FUNCTION_MODULE_NAME)
        module = module_from_spec(spec)
        spec.loader.exec_module(module)
        return module.model_loader
    
    def _get_budget_to_data(self) -> Callable[Budget, xr.Dataset]:
        """
        Get the mapping from budget keys to data keys
        """
        from importlib.util import spec_from_file_location, module_from_spec
        spec = spec_from_file_location(self._FUNCTION_MODULE_NAME.replace(".py", ""), self.model_path / self._FUNCTION_MODULE_NAME)
        module = module_from_spec(spec)
        spec.loader.exec_module(module)
        return module.budget_to_data
    
    def predict(
        self, 
        budget: Budget # Budget
        ) -> xr.DataArray: # Predicted target variable
        """
        Predict the target variable from the input data
        """
        data = self._budget_to_data(budget, self._model)
        return self._model.predict(data)
    
    
    def contributions(
        self, 
        budget: Budget # Budget
        ) -> xr.Dataset: # Contributions of the input data to the target variable
        """
        Get the contributions of the input data to the target variable
        """
        data = self._budget_to_data(budget, self._model)
        return self._model.contributions(data)

In [ ]:
class BudgetModel(BaseBudgetModel):
    """
    Budget model class
    """
    ...

In [ ]:
m = BudgetModel("Revenue Model", "Revenue", "../../example_files/model_1")

In [ ]:
show_doc(BaseBudgetModel.predict)

---

[source](https://github.com/redam94/budget_optimizer/blob/main/budget_optimizer/utils/model_classes.py#L51){target="_blank" style="float:right; font-size:smaller"}

### BaseBudgetModel.predict

>      BaseBudgetModel.predict (budget:Budget)

*Predict the target variable from the input data*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| budget | Budget | Budget |
| **Returns** | **DataArray** | **Predicted target variable** |

In [ ]:
m.predict({"a": 2, "b": .3})

<xarray.DataArray 'prediction' (time: 3)> Size: 24B
array([3.2, 5.5, 7.8])
Coordinates:
  * time     (time) int64 24B 1 2 3

In [ ]:
show_doc(BaseBudgetModel.contributions)

---

[source](https://github.com/redam94/budget_optimizer/blob/main/budget_optimizer/utils/model_classes.py#L61){target="_blank" style="float:right; font-size:smaller"}

### BaseBudgetModel.contributions

>      BaseBudgetModel.contributions (budget:Budget)

*Get the contributions of the input data to the target variable*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| budget | Budget | Budget |
| **Returns** | **Dataset** | **Contributions of the input data to the target variable** |

In [ ]:
m.contributions({"a": 2, "b": .3})

<xarray.Dataset> Size: 72B
Dimensions:  (time: 3)
Coordinates:
  * time     (time) int64 24B 1 2 3
Data variables:
    a        (time) int64 24B 2 4 6
    b        (time) float64 24B 1.2 1.5 1.8

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()